In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle
import numpy as np
import tensorflowjs as tfjs
from keras.callbacks import EarlyStopping

#Load training data
X = pickle.load(open("XEdgyRound.pickle", "rb"))
Y = pickle.load(open("YEdgyRound.pickle", "rb"))

#normalize input values (pixel values range from 0 - 255)
X = np.array(X/255.0)
Y = np.array(Y)

#Load validation data
ValidX = pickle.load(open("XEdgyRoundValid.pickle", "rb"))
ValidY = pickle.load(open("YEdgyRoundValid.pickle", "rb"))

#normalize input values (pixel values range from 0 - 255)
ValidX = np.array(ValidX/255.0)
ValidY = np.array(ValidY)


#Build the architecture of the network:

model = Sequential()
model.add(Conv2D(32, (3,3), padding="same", input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(64, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(128, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2,2)))


model.add(Conv2D(256, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2,2)))


model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Dropout(0.5))


model.add(Dense(2))
model.add(Activation("softmax"))

#Use Adam optimizer and, if desired, specify learning rate:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
optimizer = tf.keras.optimizers.Adam()

#Select loss function:
model.compile(loss = "binary_crossentropy", 
              optimizer = optimizer,
              metrics = ['accuracy'])

#stop early if loss does not further decrease after a specified number of epochs (patience):
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=8)

#print a summary of model:
model.summary()

#train model with specified batch size for a maximum specified number of epochs
model.fit(X, Y, batch_size = 64, epochs = 50, validation_data = (ValidX, ValidY), callbacks = [es])

#save the model:
model.save('MODELNAME')

#convert model to a format that is readable by tensorflow.js to use it in the word generator:
tfjs.converters.save_keras_model(model, "SAVE PATH")

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 100, 100, 32)      320       
_________________________________________________________________
activation_10 (Activation)   (None, 100, 100, 32)      0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 50, 50, 64)        18496     
_________________________________________________________________
activation_11 (Activation)   (None, 50, 50, 64)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 25, 25, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 25, 25, 128)      